## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-21-19-39-27 +0000,business,FAA investigating after Delta plane from Minne...,https://www.startribune.com/delta-b52-near-col...
1,2025-07-21-19-27-06 +0000,business,Mass vandalism and thefts from vehicles spread...,https://www.startribune.com/mass-vandalism-and...
2,2025-07-21-18-59-50 +0000,business,‘It’s good to be alive’: Sen. John Hoffman spe...,https://www.startribune.com/its-good-to-be-ali...
3,2025-07-21-18-35-00 +0000,business,Sen. Nicole Mitchell to resign seat by early A...,https://www.startribune.com/sen-nicole-mitchel...
4,2025-07-21-17-03-07 +0000,business,Sun Street Breads bakery closing in March: ‘It...,https://www.startribune.com/sun-street-breads-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2285/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
4,minneapolis,3
18,sen,2
73,up,2
47,minnesota,2
31,seat,2


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
7,2025-07-21-16-27-27 +0000,business,Republican Eric Pratt will run for open congre...,https://www.startribune.com/state-sen-eric-pra...,14
9,2025-07-21-13-04-03 +0000,business,Roper: DFL endorsement of democratic socialist...,https://www.startribune.com/minneapolis-dfl-co...,14
2,2025-07-21-18-59-50 +0000,business,‘It’s good to be alive’: Sen. John Hoffman spe...,https://www.startribune.com/its-good-to-be-ali...,13
0,2025-07-21-19-39-27 +0000,business,FAA investigating after Delta plane from Minne...,https://www.startribune.com/delta-b52-near-col...,11
3,2025-07-21-18-35-00 +0000,business,Sen. Nicole Mitchell to resign seat by early A...,https://www.startribune.com/sen-nicole-mitchel...,11


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
7,14,2025-07-21-16-27-27 +0000,business,Republican Eric Pratt will run for open congre...,https://www.startribune.com/state-sen-eric-pra...
9,14,2025-07-21-13-04-03 +0000,business,Roper: DFL endorsement of democratic socialist...,https://www.startribune.com/minneapolis-dfl-co...
2,13,2025-07-21-18-59-50 +0000,business,‘It’s good to be alive’: Sen. John Hoffman spe...,https://www.startribune.com/its-good-to-be-ali...
5,10,2025-07-21-16-59-00 +0000,business,Inside a tiny Minnesota town’s fight to save i...,https://www.startribune.com/inside-a-tiny-minn...
6,9,2025-07-21-16-31-19 +0000,business,RandBall: What if the geniuses running the Twi...,https://www.startribune.com/twins-derek-falvey...
4,9,2025-07-21-17-03-07 +0000,business,Sun Street Breads bakery closing in March: ‘It...,https://www.startribune.com/sun-street-breads-...
0,8,2025-07-21-19-39-27 +0000,business,FAA investigating after Delta plane from Minne...,https://www.startribune.com/delta-b52-near-col...
1,7,2025-07-21-19-27-06 +0000,business,Mass vandalism and thefts from vehicles spread...,https://www.startribune.com/mass-vandalism-and...
3,7,2025-07-21-18-35-00 +0000,business,Sen. Nicole Mitchell to resign seat by early A...,https://www.startribune.com/sen-nicole-mitchel...
8,3,2025-07-21-15-01-00 +0000,business,How to sign up for Minnesota’s 2025 e-bike rebate,https://www.startribune.com/how-to-sign-up-for...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
